In [ ]:
# precs, recall (stem)
# rougle, meteor, bleu 4, EM, 

In [3]:
import os
import numpy as np
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import  Cider
import bleu

def calc_prec(pred, tgt):
    prds = pred.split()
    tgs = tgt.split()
    count = 0 
    for prd in prds:
        if prd in tgs:
            count += 1
    return count/ len(prds)

def cal_metrics(index):
    prd_dir = os.path.join(f'{output_dir}',f'test.output')
    gold_dir = os.path.join(f'{output_dir}',f'test.gold')
    # prd_dir = os.path.join(f'{output_dir}','test_best-bleu.output')
    # gold_dir = os.path.join(f'{output_dir}','test_best-bleu.gold')

    # prd_dir = os.path.join(f'{output_dir}','preds.txt')
    # gold_dir = os.path.join(f'{output_dir}','msg.txt')
    # prd_dir = os.path.join(f'{output_dir}','test_0.output')
    # gold_dir = os.path.join(f'{output_dir}','test_0.gold')
    # prd_dir = os.path.join(f'{output_dir}','test.output')
    # gold_dir = os.path.join(f'{output_dir}','test.gold')
    predictions = list()
    golds = list()
    
    with open(prd_dir) as f:
        predictions =  [line.strip() for line in f.readlines()]
    with open(gold_dir) as f:
        golds =  [line.strip() for line in f.readlines()]
    # with open(prd_dir) as f:
    #     predictions =  [str(i)+ '\t' + line.strip() for i,line in enumerate(f.readlines())]
    # with open(gold_dir) as f:
    #     golds =  [str(i)+ '\t' + line.strip() for i,line in enumerate(f.readlines())]
    tmp_file= 'tmpgold.txt'
    with open(tmp_file,'w+') as f:
        f.write('\n'.join(golds))
    
    EM = list()
    precs = list()
    for i, (ref, gold) in enumerate(zip(predictions, golds)):
        EM.append(ref.split() == gold.split())
        precs.append(calc_prec(ref,gold))
    (goldMap, predictionMap) = bleu.computeMaps(
        predictions, tmp_file)
    dev_bleu = round(bleu.bleuFromMaps(goldMap, predictionMap)[0], 3)
    (goldMap2, predictionMap2) = bleu.computeMaps(
        predictions, tmp_file)
    dev_bleu2 = round(bleu.bleuFromMaps(goldMap2, predictionMap2)[0], 3)
    EM = round(np.mean(EM)*100, 3)
    precs = round(np.mean(precs)*100, 3)
    res = {k: [' '.join(v.split('\t')[1:]).strip().lower()] for k, v in enumerate(predictions)}
    tgt = {k: [' '.join(v.split('\t')[1:]).strip().lower()] for k, v in enumerate(golds)}
    
    # precision 1-gram
    print(len(res))
    print(len(tgt))
    score_Meteor, scores_Meteor = Meteor().compute_score(tgt, res)
    print("Meteor: %s" % (float(score_Meteor)*100))
    score_Rouge, scores_Rouge = Rouge().compute_score(tgt, res)
    print("ROUGE: %s" % (float(score_Rouge)*100))
    print(" %s = %s " % ("EM", str(EM)))
    print(" %s = %s " % ("precs", str(precs)))
    print("  %s = %s " % ("bleu-4", str(dev_bleu)))
    print("  %s = %s " % ("bleu-normal", str(dev_bleu2)))
    os.remove(tmp_file)
tt = 't1002'
# output_dir = 'change_size_stt'
output_dir = 'output/commitbart'
for i in range(1,2):
    print(i)
    cal_metrics(i)
!for pid in $(ps -ef | grep meteor | awk {'print $2'}); do kill -9 $pid; done

1


Total: 3040
Total: 3040


3040
3040
Meteor: 16.99408145616588
ROUGE: 24.628769788084607
 EM = 4.013 
 precs = 33.907 
  bleu-4 = 22.782 
  bleu-normal = 22.782 


In [ ]:
# recall presc

In [4]:
cache_stem = dict()
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
ps = PorterStemmer()
import pandas as pd 
with open(f'{output_dir}/test.output') as f:
    out = [l.strip() for l in f.readlines()]
with open(f'{output_dir}/test.gold') as f:
    gold = [l.strip() for l in f.readlines()]

In [16]:
def preprocess(sentence):
    sentence = sentence.lower()
    sentence = [ps.stem(l) for l in sentence.split()]
    print(sentence)
    return sentence
def calc(ref,pred):
    ref = preprocess(ref)
    pred = preprocess(pred)
    inter = set(ref).intersection(set(pred))
    recall = len(inter) / len(ref)
    precs = len(inter) / len(pred)
    # doan dung / tong so dung recal
    # doan dung / tong so doan prec
    return recall, precs
# read 

In [7]:
df = pd.DataFrame()
df['out'] = out 
df['ref'] = gold

In [ ]:
recalls = list()
precisions = list()
for _,row in df.iterrows():
    r,p = calc(row['ref'],row['out'])
    recalls.append(r)
    precisions.append(p)
df['recall'] = recalls
df['precision'] = precisions

In [53]:
import json
codes = list()
file_old = list()
file_new = list()
cms = list()
repos = list()
with open('test.jsonl') as f:
    for l in f.readlines()[:3040]:
        data = json.loads(l.strip())
        msg = data['message']
        cms.append(data['commit_id'])
        # repos.append(data['repo'])
        
        code = list()
        old = list()
        new = list()
        for diff in data['diffs']:
            old.append(diff['negative_changed_file_name'])
            new.append(diff['positive_changed_file_name'])
            for chunk in diff['chunks']:
                code.append(chunk['chunk_str'])
        codes.append('\n'.join(code))
        file_old.append('\n'.join(old))
        file_new.append('\n'.join(new))
len(codes)

3040

In [55]:
df['code'] = codes
df['file_new'] = file_new
df['file_old'] = file_old
df['commit_id'] = cms
# df['repo'] = repos

In [37]:
df.to_excel('result.xlsx')

In [38]:
sum(df['recall'])/ df.shape[0], sum(df['precision'])/ df.shape[0]

(0.3615022193230006, 0.3643810049862839)